# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

1. Znajdź listę wszystkich filmów o tej samej długości.

z uwagi na brak jakiegokolwiek pomysłu co może oznaczać to pytanie poza listą list filmów o zadanej długości (co nie do końca mnie przekonuje). to pytanie dla mnie brzmi "znajdź listę liczby filmów w poszczególnych zbiorach dla których zdefiniowanych dla wszystkich czasów z wyłączeniem zbiorów pustych"

In [1]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');


In [2]:
strr = f"""
SELECT
    film.length, COUNT(film.film_id)
FROM
    film
GROUP BY
    film.length
"""
df = pd.read_sql(strr, con=connection)
print(df)

     length  count
0       116      4
1       184      8
2        87      6
3        71      7
4        68      5
..      ...    ...
135     148      7
136     129      7
137     130      6
138     143      7
139      58      7

[140 rows x 2 columns]


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\2758686133.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

podobnie jak wyżej tylko długość na miasta

In [3]:
strr = f"""
SELECT
    city.city, COUNT(customer.customer_id)
FROM
    customer, address,city
WHERE
    customer.address_id = address.address_id
    AND address.city_id = city.city_id
GROUP BY
    city.city
"""
df = pd.read_sql(strr, con=connection)
print(df)

                city  count
0          Southport      1
1             Taguig      1
2              Tokat      1
3            Atlixco      1
4           Mukateve      1
..               ...    ...
592           Warren      1
593        Pingxiang      1
594       Greensboro      1
595  Ocumare del Tuy      1
596         Pemalang      1

[597 rows x 2 columns]


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\3926990307.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
strr = f"""
SELECT
    AVG(film.rental_rate)
FROM
    film
"""
df = pd.read_sql(strr, con=connection)
print(df)

    avg
0  2.98


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\3937777551.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
strr = f"""
SELECT
    category.name AS category ,COUNT(film.title)
FROM
    category, film_category, film
WHERE
    category.category_id = film_category.category_id
    AND film.film_id = film_category.film_id
GROUP BY
    category.name
"""
df = pd.read_sql(strr, con=connection)
print(df)

       category  count
0        Sports     74
1      Classics     57
2           New     63
3        Family     69
4        Comedy     58
5     Animation     66
6        Travel     57
7         Music     51
8         Drama     62
9        Horror     56
10       Sci-Fi     61
11        Games     61
12  Documentary     68
13      Foreign     73
14       Action     64
15     Children     60


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\3308867546.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
strr = f"""
SELECT
    country.country ,COUNT(customer.customer_id)
FROM
    country, city, address, customer
WHERE
    country.country_id = city.country_id
    AND address.city_id = city.city_id
    AND customer.address_id = address.address_id
GROUP BY
    country.country
"""
df = pd.read_sql(strr, con=connection)
print(df)

                  country  count
0              Bangladesh      3
1               Indonesia     14
2               Venezuela      7
3                Cameroon      2
4          Czech Republic      1
..                    ...    ...
103              Tanzania      3
104                Poland      8
105             Greenland      1
106         French Guiana      1
107  Virgin Islands, U.S.      1

[108 rows x 2 columns]


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\322804802.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
strr = f"""
SELECT
    customer.store_id ,COUNT(customer.customer_id)
FROM
    customer
GROUP BY
    customer.store_id
HAVING
   COUNT(customer.customer_id)>=100
   AND COUNT(customer.customer_id) <= 300;
"""
df = pd.read_sql(strr, con=connection)
print(df)

   store_id  count
0         2    273


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\4122560733.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
strr = f"""
SELECT
    customer.first_name, customer.last_name, SUM(film.length)
FROM
    customer,rental,inventory,film
WHERE
    customer.customer_id = rental.customer_id
    AND rental.inventory_id = inventory.inventory_id
    AND inventory.film_id = film.film_id
GROUP BY
    customer.customer_id
HAVING
   SUM(film.length)>=200
ORDER BY
    SUM(film.length)
"""
df = pd.read_sql(strr, con=connection)
print(df)

C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\3717042686.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


    first_name last_name   sum
0        Brian     Wyman  1265
1      Antonio      Meek  1451
2        Leona    Obrien  1588
3    Katherine    Rivera  1615
4      Tiffany    Jordan  1667
..         ...       ...   ...
594     Wesley      Bull  4808
595      Clara      Shaw  4808
596      Tammy   Sanders  5065
597    Eleanor      Hunt  5360
598       Karl      Seal  5388

[599 rows x 3 columns]


8. Oblicz średnią wartość wypożyczenia filmu.

In [9]:
strr = f"""
SELECT
    AVG(payment.amount)
FROM
    payment
"""
df = pd.read_sql(strr, con=connection)
print(df)

        avg
0  4.200606


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\2616645804.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
strr = f"""
SELECT
    category.name AS category ,AVG(film.length)
FROM
    category, film_category, film
WHERE
    category.category_id = film_category.category_id
    AND film.film_id = film_category.film_id
GROUP BY
    category.name
"""
df = pd.read_sql(strr, con=connection)
print(df)

       category         avg
0        Sports  128.202703
1      Classics  111.666667
2           New  111.126984
3        Family  114.782609
4        Comedy  115.827586
5     Animation  111.015152
6        Travel  113.315789
7         Music  113.647059
8         Drama  120.838710
9        Horror  112.482143
10       Sci-Fi  108.196721
11        Games  127.836066
12  Documentary  108.750000
13      Foreign  121.698630
14       Action  111.609375
15     Children  109.800000


C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\118076861.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
strr = f"""
SELECT
    new_tab.category, film.title
FROM(
    SELECT
        category.name AS category ,MAX(LENGTH(film.title))
    FROM
        category, film_category, film
    WHERE
        category.category_id = film_category.category_id
        AND film.film_id = film_category.film_id
    GROUP BY
        category.name
    ) new_tab, category, film_category, film
WHERE
    category.category_id = film_category.category_id
    AND film.film_id = film_category.film_id
    AND new_tab.category = category.name
    AND new_tab.MAX = LENGTH(film.title)
ORDER BY
    new_tab.category


"""

df = pd.read_sql(strr, con=connection)
print(df)

       category                        title
0        Action      Entrapment Satisfaction
1     Animation       Telemark Heartbreakers
2      Children         Heartbreakers Bright
3      Children         Microcosmos Paradise
4      Children         Sweethearts Suspects
5      Classics      Extraordinary Conquerer
6        Comedy      Trainspotting Strangers
7   Documentary       Intolerable Intentions
8   Documentary       Deliverance Mulholland
9         Drama       Goldfinger Sensibility
10       Family       Resurrection Silverado
11      Foreign         Reservoir Adaptation
12      Foreign         Impossible Prejudice
13      Foreign         Ballroom Mockingbird
14        Games        Creatures Shakespeare
15        Games        Bulworth Commandments
16       Horror  Arachnophobia Rollercoaster
17        Music       Confidential Interview
18          New        Flamingos Connecticut
19       Sci-Fi         Cincinatti Whisperer
20       Sci-Fi         Silverado Goldfinger
21       S

C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\1531461001.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [12]:
strr = f"""
SELECT
    new_tab.category, film.title
FROM(
    SELECT
        category.name AS category ,MAX(film.length)
    FROM
        category, film_category, film
    WHERE
        category.category_id = film_category.category_id
        AND film.film_id = film_category.film_id
    GROUP BY
        category.name
    ) new_tab, category, film_category, film
WHERE
    category.category_id = film_category.category_id
    AND film.film_id = film_category.film_id
    AND new_tab.category = category.name
    AND new_tab.MAX = film.length
ORDER BY
    new_tab.category
"""
df = pd.read_sql(strr, con=connection)
print(df)

C:\Users\nikos\AppData\Local\Temp\ipykernel_30516\746609497.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


       category               title
0        Action      Darn Forrester
1        Action        Worst Banger
2     Animation        Pond Seattle
3     Animation         Gangs Pride
4      Children         Fury Murder
5      Children      Wrong Behavior
6      Classics   Conspiracy Spirit
7        Comedy      Control Anthem
8   Documentary      Young Language
9   Documentary           Wife Turn
10        Drama       Jacket Frisco
11       Family      King Evolution
12      Foreign      Sorority Queen
13      Foreign    Crystal Breaking
14        Games       Chicago North
15       Horror    Analyze Hoosiers
16       Horror       Love Suicides
17        Music           Home Pity
18          New      Frontier Cabin
19       Sci-Fi  Soldiers Evolution
20       Sports     Smoochy Control
21       Travel   Sweet Brotherhood
22       Travel       Muscle Bright


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [14]:
import main

#connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');
df = main.film_in_category(1)
print(df)

                     title               languge category
0             Amadeus Holy  English                Action
1          American Circus  English                Action
2       Antitrust Tomatoes  English                Action
3            Ark Ridgemont  English                Action
4      Barefoot Manchurian  English                Action
..                     ...                   ...      ...
59         Uprising Uptown  English                Action
60  Waterfront Deliverance  English                Action
61           Werewolf Lola  English                Action
62            Women Dorado  English                Action
63            Worst Banger  English                Action

[64 rows x 3 columns]


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [15]:
df = main.number_films_in_category(1)
print(df)

  category  count
0   Action     64


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [23]:
df = main.number_film_by_length(80,88)
print(df)

   length  count
0      80     10
1      81      3
2      82      7
3      83      6
4      84     13
5      85     17
6      86      5
7      87      6
8      88      4


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [18]:
df = main.client_from_city("London")
print(df)

     city first_name last_name
0  London     Mattie   Hoffman
1  London      Cecil     Vines


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [19]:
df = main.avg_amount_by_length(48)
print(df)

   length       avg
0      48  4.295389


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:187: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [20]:
df = main.client_by_sum_length(1264)
print(df)

    first_name last_name   sum
0        Brian     Wyman  1265
1      Antonio      Meek  1451
2        Leona    Obrien  1588
3    Katherine    Rivera  1615
4      Tiffany    Jordan  1667
..         ...       ...   ...
594     Wesley      Bull  4808
595      Clara      Shaw  4808
596      Tammy   Sanders  5065
597    Eleanor      Hunt  5360
598       Karl      Seal  5388

[599 rows x 3 columns]


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:227: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)


In [21]:
df = main.category_statistic_length("Action")
print(df)

  category         avg   sum  min  max
0   Action  111.609375  7143   47  185


c:\D\AIBD\AIBD_Michal_Michniak-main\lab2_join-MichalMichniak-master\main.py:261: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strr, con=connection)
